In [1]:
# utils/llm.py

import os

from dotenv import load_dotenv

In [2]:
from langchain_core.prompts import PromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_pinecone import PineconeVectorStore


c:\Users\1-06\Desktop\team4-backend\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

ImportError: cannot import name 'InputTokenDetails' from 'langchain_core.messages.ai' (c:\Users\1-06\Desktop\team4-backend\venv\Lib\site-packages\langchain_core\messages\ai.py)